In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from energy_consumption.help_functions.get_energy_data import get_data, set_last_hour
from energy_consumption.models.ec_model1 import get_ec_forecasts_model1

In [ ]:
start_date_excl = most_recent_thursday(df)
df_cval = df.loc[df.index < start_date_excl]

In [4]:
import requests
from datetime import datetime, timedelta
from tqdm import tqdm

In [14]:
last_x = 1
df_before, df_after = split_time(energydata, num_weeks=1)

pred = get_ec_forecasts_model1(df_before)
pred

C:\Users\Maria\Documents\Studium\Pyhton Projekte\PTSFC\energy_consumption\help_functions\dummy_mapping.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['month'] = data_df.index.month
C:\Users\Maria\Documents\Studium\Pyhton Projekte\PTSFC\energy_consumption\help_functions\dummy_mapping.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[season] = data_df['month'].apply(
C:\Users\Maria\Documents\Studium\Pyhton Projekte\PTSFC\energy_consumption\help_functions\dummy_mapping.py:18: SettingWit

,date_time,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-08 13:00:00,2023-11-14,energy,36 hour,50.687106,60.204249,62.799751,65.326362,73.36025
1,2023-11-08 17:00:00,2023-11-14,energy,40 hour,50.065606,58.139620,60.382238,63.194486,72.61725
2,2023-11-08 21:00:00,2023-11-14,energy,44 hour,48.004605,54.453731,55.709061,57.331219,63.90400
3,2023-11-09 13:00:00,2023-11-14,energy,60 hour,50.687106,60.204249,62.799751,65.326362,73.36025
4,2023-11-09 17:00:00,2023-11-14,energy,64 hour,50.065606,58.139620,60.382238,63.194486,72.61725
5,2023-11-09 21:00:00,2023-11-14,energy,68 hour,48.004605,54.453731,55.709061,57.331219,63.90400


In [19]:
model1 = {
    'name': 'model1',
    'function': get_ec_forecasts_model1
}

last_x = 1
df_before, df_after = split_time(energydata, num_weeks=10)

pred = model1['function'](df_before)
pred

C:\Users\Maria\Documents\Studium\Pyhton Projekte\PTSFC\energy_consumption\help_functions\dummy_mapping.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['month'] = data_df.index.month
C:\Users\Maria\Documents\Studium\Pyhton Projekte\PTSFC\energy_consumption\help_functions\dummy_mapping.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[season] = data_df['month'].apply(
C:\Users\Maria\Documents\Studium\Pyhton Projekte\PTSFC\energy_consumption\help_functions\dummy_mapping.py:18: SettingWit

,date_time,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-09-06 13:00:00,2023-11-14,energy,36 hour,52.100127,61.698569,64.606318,67.814389,73.74275
1,2023-09-06 17:00:00,2023-11-14,energy,40 hour,51.035499,59.581610,62.551749,65.827639,72.78350
2,2023-09-06 21:00:00,2023-11-14,energy,44 hour,49.285999,56.116968,57.632180,59.918914,64.41875
3,2023-09-07 13:00:00,2023-11-14,energy,60 hour,52.100127,61.698569,64.606318,67.814389,73.74275
4,2023-09-07 17:00:00,2023-11-14,energy,64 hour,51.035499,59.581610,62.551749,65.827639,72.78350
5,2023-09-07 21:00:00,2023-11-14,energy,68 hour,49.285999,56.116968,57.632180,59.918914,64.41875


In [29]:
# Calculate score for single quantile prediction 
def evaluate_quantile(sq_pred, tau, sy_obs):
    l_score = None
    if sq_pred > sy_obs:
        l_score = 2 * (1 - tau) * (sq_pred - sy_obs)
    else:
        l_score = 2 * tau * (sy_obs - sq_pred)
    return l_score

# Calculate score for all quantile predictions
def evaluate_horizon(quantilepreds, obs):
    taus = [0.025, 0.25, 0.5, 0.75, 0.975]
    if len(quantilepreds) != 5:
        raise ValueError("The quantiles must consist of five elements.")
    total_sum = 0
    for q, tau in zip(quantilepreds, taus):
        l_score = evaluate_quantile(q, tau, obs)
        total_sum += l_score
    return total_sum

In [30]:
merged_df = pd.merge(energydata, pred, on='date_time', how='inner')
merged_df


,date_time,energy_consumption,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-09-06 13:00:00,61.25900,energy,36 hour,52.100127,61.698569,64.606318,67.814389,73.74275
1,2023-09-06 17:00:00,56.34750,energy,40 hour,51.035499,59.581610,62.551749,65.827639,72.78350
2,2023-09-06 21:00:00,53.38100,energy,44 hour,49.285999,56.116968,57.632180,59.918914,64.41875
3,2023-09-07 13:00:00,61.38050,energy,60 hour,52.100127,61.698569,64.606318,67.814389,73.74275
4,2023-09-07 17:00:00,58.91425,energy,64 hour,51.035499,59.581610,62.551749,65.827639,72.78350
5,2023-09-07 21:00:00,54.32175,energy,68 hour,49.285999,56.116968,57.632180,59.918914,64.41875


In [37]:
for index, rowcontent in merged_df.iterrows():
    quantile_preds = rowcontent[[
       'q0.025', 'q0.25', 'q0.5', 'q0.75', 'q0.975']]
    observation = rowcontent['energy_consumption']
    score = evaluate_horizon(quantile_preds, observation)
    merged_df.at[index, 'score'] = score

merged_df

,date_time,energy_consumption,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975,score
0,2023-09-06 13:00:00,61.25900,energy,36 hour,52.100127,61.698569,64.606318,67.814389,73.74275,8.366496
1,2023-09-06 17:00:00,56.34750,energy,40 hour,51.035499,59.581610,62.551749,65.827639,72.78350,16.882884
2,2023-09-06 21:00:00,53.38100,energy,44 hour,49.285999,56.116968,57.632180,59.918914,64.41875,12.380727
3,2023-09-07 13:00:00,61.38050,energy,60 hour,52.100127,61.698569,64.606318,67.814389,73.74275,8.001996
4,2023-09-07 17:00:00,58.91425,energy,64 hour,51.035499,59.581610,62.551749,65.827639,72.78350,9.182634
5,2023-09-07 21:00:00,54.32175,energy,68 hour,49.285999,56.116968,57.632180,59.918914,64.41875,9.558477


In [ ]:
last_x = 100 

for w in range(last_x):
        df_before, df_after = split_time(
            df_before, num_weeks=1)
        pred = get_ec_forecasts_model1(df_before)
        obs = pd.DataFrame({'energy_consumption': df.loc[pred['forecast_date']]["energy_consumption"]})
        pred = pred.set_index('forecast_date')
        merged_df = pd.merge(pred, obs, left_index=True, right_index=True)

        # Add scores to the merged_df
        for index, row in merged_df.iterrows():
            quantile_preds = row[[
                'q0.025', 'q0.25', 'q0.5', 'q0.75', 'q0.975']]
            observation = row['energy_consuption']
            score = evaluate_horizon(quantile_preds, observation)
            merged_df.at[index, 'score'] = score
        # print(merged_df[['q0.025','q0.25','q0.5','q0.75','q0.975']])

        evaluation = pd.concat([evaluation, merged_df])
    return evaluation